In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

In [2]:
from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataset, CHBMITDataModule

# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

2025-05-09 05:21:19.117 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /workspace/snn-encoder-test


In [3]:
datamodule = CHBMITDataModule(dataset, batch_size=256, worker=0)

In [4]:
import optuna
from loguru import logger

import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback

from eeg_snn_encoder.encoders import PhaseEncoderExpand
from eeg_snn_encoder.callback import TrackBestMetric
from eeg_snn_encoder.models.classifier import ModelConfig
from eeg_snn_encoder.models.lightning import LitSeizureClassifier, OptimizerConfig
import torch

torch.set_float32_matmul_precision('medium')


def objective(trial: optuna.Trial) -> float:
    model_params: ModelConfig = {
        "threshold": trial.suggest_float("threshold", 0.01, 0.5),
        "slope": trial.suggest_float("slope", 1.0, 20.0),
        "beta": trial.suggest_float("beta", 0.1, 0.99),
        "dropout_rate1": trial.suggest_float("dropout_rate1", 0.1, 0.99),
        "dropout_rate2": trial.suggest_float("dropout_rate2", 0.1, 0.99),
    }

    optimizer_params: OptimizerConfig = {
        "lr": trial.suggest_float("lr", 1e-6, 1e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True),
        "scheduler_factor": trial.suggest_float("scheduler_factor", 0.1, 0.99),
        "scheduler_patience": trial.suggest_int("scheduler_patience", 1, 10),
    }

    encoder_params = {
        "phase_window": trial.suggest_int("phase_window", 1, 4),
    }

    spike_encoder = PhaseEncoderExpand(**encoder_params)
    
    lit_model = LitSeizureClassifier(
        model_config=model_params,
        optimizer_config=optimizer_params,
        spike_encoder=spike_encoder,
    )

    metric_tracker = TrackBestMetric(
        monitor="val_mse",
        mode="min",
    )

    trainer = pl.Trainer(
        max_epochs=20,
        accelerator="auto",
        devices="auto",
        strategy="auto",
        enable_model_summary=False,
        enable_checkpointing=False,
        callbacks=[metric_tracker, PyTorchLightningPruningCallback(trial, monitor="val_mse"), EarlyStopping(monitor="val_loss", mode="min", patience=5)],
        logger=False,
    )

    trainer.fit(lit_model, datamodule=datamodule)

    return metric_tracker.best_metric

In [7]:
import os

# Initialize the Optuna study
sampler = optuna.samplers.TPESampler(multivariate=True, seed=47)
pruner = optuna.pruners.HyperbandPruner()
study = optuna.create_study(
    direction="minimize",
    study_name="model-tuning-pe-mse-1",
    storage=os.environ["OPTUNA_CONN_STRING"],
    load_if_exists=True,
    sampler=sampler,
    pruner=pruner,
)

/workspace/snn-encoder-test/.venv/lib/python3.12/site-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-05-09 05:26:57,119] A new study created in RDB with name: model-tuning-pe-mse-1


In [8]:
study.optimize(objective, n_trials=50)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/workspace/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


New best val_mse: 0.494140625 | Old best: inf


/workspace/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[I 2025-05-09 06:44:33,588] Trial 49 finished with value: 0.2443559169769287 and parameters: {'threshold': 0.031032962750803958, 'slope': 15.237252281576076, 'beta': 0.8136317378409649, 'dropout_rate1': 0.5022112266551046, 'dropout_rate2': 0.6191597973435314, 'lr': 7.580547637090579e-05, 'weight_decay': 5.707209952920326e-06, 'scheduler_factor': 0.9235402429066496, 'scheduler_patience': 7, 'phase_window': 2}. Best is trial 37 with value: 0.2018592357635498.


In [ ]:
logger.info(f"Encoder: Phase Encoding,trial, best_param: {study.best_params}")
logger.info(f"Encoder: Phase Encoding,trial, best_score: {study.best_value}")